# Examining _Danish Energy Agency Technology Catalogue_ energy storage data for Mopo WP5

This Jupyter Notebook aims to examine the Danish Energy Agency
energy storage technology catalogue data in order to extract it for Mopo WP5.

## Julia Setup

This script is written in Julia, and a little bit of setup is required for it to run correclty.

In [ ]:
## Activate (and set up) the required Julia environment

using Pkg # Julia package manager.
Pkg.activate(@__DIR__) # Activate the Julia environment in the folder this file is in (namely the `Project.toml`)
Pkg.instantiate() # Download and install the necessary dependencies.

# Load dependencies
using XLSX
using DataFrames

## Read an examine the raw excel data

In [ ]:
## Read the energy storage dataheets

raw_data = DataFrame(
    XLSX.readtable(
        "input-data\\dea-technology-catalogues\\technology_datasheet_for_energy_storage.xlsx",
        "alldata_flat"
    )
)
describe(raw_data)

In [ ]:
## Let's look at the parameters individually, `ws` first

Set(raw_data[!, :ws])

In [ ]:
Set(raw_data[!, :Technology])

In [ ]:
Set(raw_data[!, :cat])

In [ ]:
params = Set(raw_data[!, :par])

In [ ]:
Set(raw_data[!, :unit])

In [ ]:
Set(raw_data[!, :priceyear])

In [ ]:
Set(raw_data[!, :est])

In [ ]:
Set(raw_data[!, :year])

So we don't have that many different technologies this time,
but a ton of different parameters and units for said parameters.
There doesn't seem to be any major misspellings that affect our processing,
although the spread of different years is a bit surprising.

## Finding the desired parameters

Based on the WP5 parameter excel, the parameters of interest are:
- `CAPEX Energy`
- `CAPEX`
- `FOM Energy`
- `FOM`
- `VOM`
- `Charge Efficiency`
- `Discharge Efficiency`
- `Hours`
- `Lifetime`
- `Currency`
- `Metadata`
all for the years 2030, 2040, and 2050.

Not sure what the `CAPEX Energy` vs `CAPEX` division is,
maybe storage vs charging/discharging power if these are separate investments?
The `Hours` seems by far the most ill-defined parameter.
According to WP5, in this case it apparently refers to the ratio of
max energy content and max charging/discharging power.

In [ ]:
## Check investment parameters for CAPEX

filter(x -> occursin("invest", lowercase(x)), params)

# "Specific investment" mostly seems like what we need.

In [ ]:
## Investment parameters 

filter(x -> occursin("component", lowercase(x)), params)

# Costs are occasionally broken into components it seems, these could be relevant?

In [ ]:
## FOM

filter(x -> occursin("fixed", lowercase(x)), params)

# Seems straightforward, although units vary

In [ ]:
## Capacity per tank?

filter(x -> occursin("unit", lowercase(x)), params)

In [ ]:
## VOM

filter(x -> occursin("variable", lowercase(x)), params)

# Seems straightforward

In [ ]:
## Other O&M costs?

filter(x -> occursin("other o&m", lowercase(x)), params)

In [ ]:
## Efficiencies

filter(x -> occursin("efficiency", lowercase(x)), params)

# Should get us there

In [ ]:
## Hours of energy provided

filter(x -> occursin("capacity", lowercase(x)), params)

# Ratio between output capacity and energy storage capacity for one unit.

In [ ]:
## Lifetime

filter(x -> occursin("lifetime", lowercase(x)), params)

# Seems like an easy `Technical lifetime [years]`

In [ ]:
## Losses

filter(x -> occursin("loss", lowercase(x)), params)

# These could be used in conjunction with the efficiency params to estimate storage hours?

In [ ]:
## Temperatures?

filter(x -> occursin("temperature", lowercase(x)), params)

## Conclusions

Again, the _DEA Technology Catalogue_ data on energy storage seems comprehensive
enough for our purposes.
However, I'll need to check with WP5 to figure out the exact parameters that they want.
Regardless, here's a quick compilation on what the parameters will likely look like:
- `CAPEX Energy`: Likely some `Specific investment` or its `- energy component`, but the exact units will depend on the use case.
- `CAPEX`: Likely `Specific investment` or its `- capacity component` similar to above.
- `FOM Energy`: Likely `Fixed O&M [EUR/MWhCapacity/year]`, beware of `Fixed O&M [EUR/MWhCapacity/year)` misspelling!
- `FOM`: Likely `Fixed O&M [EUR/MW/year]` or similar.
- `VOM`: Easy `Variable O&M [EUR/MWh]`.
- `Charge Efficiency`: Charging/discharging efficiencies seem to be given as components of `Round trip efficiency`, e.g. `- Charge Efficiency [%]`. I can only hope they are available separately everywhere.
- `Discharge Efficiency`: Similar to above, but `- Discharge efficiency [%]`.
- `Hours of Energy`: Ratio of energy content to charge/discharge power.
- `Lifetime`: Easy `Technical lifetime [years]`.
- `Currency`: EUR 2020, the `priceyear` column contains this information.
- `Metadata`: Add citation to the DEA catalogue in question.